In [6]:
!pip install helper
import collections

import helper
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding
import pandas as pd

english = pd.read_csv('https://raw.githubusercontent.com/Kulbear/deep-learning-nano-foundation/master/DLND-language-translation/data/small_vocab_en',sep=';',names=['data'])
french = pd.read_csv('https://raw.githubusercontent.com/Kulbear/deep-learning-nano-foundation/master/DLND-language-translation/data/small_vocab_fr',sep=';',names=['data'])


def sentence_padding(x,length=None):
  return pad_sequences(x, maxlen=length, padding='post')


def inverse_to_text(logits,tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
  #Returns the indices of the maximum values along an axis.




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
def sentence_tokenize(input,in_token_index,ouput,out_token_index):
  counter = 0
  for sample in input:                                 
    for considered_word in sample.split():
      if considered_word not in in_token_index:
        in_token_index.update({considered_word : counter + 1})
        counter = counter + 1
  in_max_length = max([len(i.split()) for i in input])

  counter = 0
  for sample in ouput:                                 
    for considered_word in sample.split():
      if considered_word not in out_token_index:
        out_token_index.update({considered_word : counter + 1})
        counter = counter + 1
  out_max_length = max([len(i.split()) for i in ouput])

  max_length = max([in_max_length,out_max_length])

  features = max([max(in_token_index.values()) + 1,max(out_token_index.values()) + 1])

  in_results  = np.zeros(shape = (len(input), max_length, features))  
  out_results = np.zeros(shape = (len(ouput), max_length, features))

  for i, sample in enumerate(input): 
    for j, considered_word in list(enumerate(sample.split())):
      index = in_token_index.get(considered_word)
      in_results[i, j, index] = 1
  
  for i, sample in enumerate(ouput): 
    for j, considered_word in list(enumerate(sample.split())):
      index = out_token_index.get(considered_word)
      out_results[i, j, index] = 1. 

  #tok.fit_on_texts(lst_sentences)
  return in_results,out_results

In [23]:
eng_token_index = {}
french_token_index = {}
eng_input,french_output = sentence_tokenize(english['data'][0:1000],eng_token_index,french['data'][0:1000],french_token_index)
print(eng_input.shape)
print(french_output.shape)


(1000, 21, 263)
(1000, 21, 263)


In [24]:
def encoder_decoder(input_shape):

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers

    model = Sequential()
    (timesteps,n_features) = input_shape[1:3]
    model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=False))
    model.add(RepeatVector(timesteps))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(128, activation='relu', return_sequences=True))

    model.add(TimeDistributed(Dense(n_features)))
    model.summary() 

    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model




# TODO: Train and Print prediction(s)
embed_rnn_model = encoder_decoder(eng_input.shape)

embed_rnn_model.summary()

embed_rnn_model.fit(eng_input, french_output, batch_size=64, epochs=100, validation_split=0.2)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 21, 128)           200704    
                                                                 
 lstm_29 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_7 (RepeatVect  (None, 21, 64)           0         
 or)                                                             
                                                                 
 lstm_30 (LSTM)              (None, 21, 64)            33024     
                                                                 
 lstm_31 (LSTM)              (None, 21, 128)           98816     
                                                                 
 time_distributed_7 (TimeDis  (None, 21, 263)          33927     
 tributed)                                            